Data from https://covidtracking.com

You can read lots of caveats (including different data handling by state) at that website.

Another caveat: In order to plot on a log scale, I replaced non-positive values with 1. I don't think this changes the message much, and for the states plotted there are no such replacements since March 15.

In [1]:
import pandas as pd
import altair as alt
import datetime

# CSV for "States daily 4 pm ET" from https://covidtracking.com/api/
# downloaded around 9am ET on 21 March 2020
# Note: "All data is cumulative"
states_cumulative_df = pd.read_csv("/Users/davidchudzicki/Downloads/states-daily.csv")
states_cumulative_df.date = states_cumulative_df.date.map(lambda dt: datetime.datetime.strptime(str(dt), '%Y%m%d'))
states_cumulative_df = states_cumulative_df[['date', 'state', 'positive', 'total']]

states_cumulative_df = states_cumulative_df.sort_values(by=['state', 'date'])
states_cumulative_df = states_cumulative_df.set_index(['date', 'state'])
shifted = states_cumulative_df.groupby(level='state').shift(1)
joined = states_cumulative_df.rename(columns=lambda x: x + "_today").join(shifted.rename(columns=lambda x: x + "_yesterday"))

# new cases:
states_new_df = joined.assign(positive=joined.positive_today - joined.positive_yesterday,
                              total=joined.total_today - joined.total_yesterday)
states_new_df = states_new_df.assign(rate =states_new_df.positive / states_new_df.total)
states_new_df = states_new_df.reset_index()

In [2]:
# spot-checking these computations by comparing raw data with my computations:

# based on raw data AK totals (3/6 to 3/10) should be: 5, 6, 12, 0
# based on raw data WY totals (3/16 to 3/20) should be: 102, 88, ...
# based on raw data WY positive (3/16 to 3/20) should be: 7, 5, 3, 1
pd.options.display.max_rows = 100
states_cumulative_df

,,positive,total
date,state,,
2020-03-06,AK,0,9
2020-03-07,AK,0,14
2020-03-08,AK,0,20
2020-03-09,AK,0,32
2020-03-10,AK,0,32
...,...,...,...
2020-03-16,WY,3,3
2020-03-17,WY,10,105
2020-03-18,WY,15,193


In [3]:
states_new_df

,date,state,positive_today,total_today,positive_yesterday,total_yesterday,positive,total,rate
0,2020-03-06,AK,0,9,NaN,NaN,NaN,NaN,NaN
1,2020-03-07,AK,0,14,0.0,9.0,0.0,5.0,0.000000
2,2020-03-08,AK,0,20,0.0,14.0,0.0,6.0,0.000000
3,2020-03-09,AK,0,32,0.0,20.0,0.0,12.0,0.000000
4,2020-03-10,AK,0,32,0.0,32.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...
808,2020-03-16,WY,3,3,3.0,3.0,0.0,0.0,NaN
809,2020-03-17,WY,10,105,3.0,3.0,7.0,102.0,0.068627
810,2020-03-18,WY,15,193,10.0,105.0,5.0,88.0,0.056818
811,2020-03-19,WY,18,289,15.0,193.0,3.0,96.0,0.031250


In [4]:
# Some more spot-checking my computations
# raw:

states_cumulative_df.groupby('state').apply(lambda df: df.sort_values(by='date')[:3])

positive  total
state date       state                 
AK    2020-03-06 AK            0      9
      2020-03-07 AK            0     14
      2020-03-08 AK            0     20
AL    2020-03-07 AL            0      0
      2020-03-08 AL            0      0
...                          ...    ...
WV    2020-03-07 WV            0      5
      2020-03-08 WV            0      5
WY    2020-03-07 WY            0      0
      2020-03-08 WY            0      0
      2020-03-09 WY            0      0

[168 rows x 2 columns]

In [5]:
# mine:

states_new_df.groupby('state').apply(lambda df: df.sort_values(by='date')[:2])

date state  positive_today  total_today  positive_yesterday  \
state                                                                         
AK    0   2020-03-06    AK               0            9                 NaN   
      1   2020-03-07    AK               0           14                 0.0   
AL    15  2020-03-07    AL               0            0                 NaN   
      16  2020-03-08    AL               0            0                 0.0   
AR    29  2020-03-06    AR               0            6                 NaN   
...              ...   ...             ...          ...                 ...   
WI    768 2020-03-05    WI               1           26                 1.0   
WV    784 2020-03-06    WV               0            5                 NaN   
      785 2020-03-07    WV               0            5                 0.0   
WY    799 2020-03-07    WY               0            0                 NaN   
      800 2020-03-08    WY               0            0                 0.0   

           total_yesterday  positive  total  rate  
state                                              
AK    0                NaN       NaN    NaN   NaN  
      1                9.0       0.0    5.0   0.0  
AL    15               NaN       NaN    NaN   NaN  
      16               0.0       0.0    0.0   NaN  
AR    29               NaN       NaN    NaN   NaN  
...                    ...       ...    ...   ...  
WI    768             26.0       0.0    0.0   NaN  
WV    784              NaN       NaN    NaN   NaN  
      785              5.0       0.0    0.0   NaN  
WY    799              NaN       NaN    NaN   NaN  
      800              0.0       0.0    0.0   NaN  

[112 rows x 9 columns]

In [6]:
population_df_raw = pd.read_csv("https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv")
population_df = population_df_raw[(population_df_raw['state/region'] != 'USA') &
                                  (population_df_raw.year == 2013) &
                                  (population_df_raw.ages == 'total')
             ].rename(columns={'state/region': 'state'})[['state', 'population']]



In [7]:

states_new_df = states_new_df.merge(population_df)
n_states = 9

top_states = list(states_new_df.groupby('state').total.sum().reset_index().sort_values(by='total', ascending=False).state[:n_states])

# replace non-positive numbers with 1 to make the log-scale chart work better
states_new_df_hacked = states_new_df.copy()
states_new_df_hacked = states_new_df_hacked.assign(hacked=False)

states_new_df_hacked.loc[(states_new_df_hacked.positive <= 0), 'hacked'] = True
states_new_df_hacked.loc[(states_new_df_hacked.positive <= 0), 'positive'] = 1
states_new_df_hacked.loc[(states_new_df_hacked.total <= 0), 'hacked'] = True
states_new_df_hacked.loc[(states_new_df_hacked.total <= 0), 'total'] = 1


per_capita = pd.DataFrame({'state': states_new_df_hacked.state, 'date': states_new_df_hacked.date,
                           'positive': states_new_df_hacked.positive / states_new_df_hacked.population,
                           'total': states_new_df_hacked.total / states_new_df_hacked.population,
                           'hacked': states_new_df_hacked.hacked
                           })


per_capita_state_subset = per_capita[per_capita.state.isin(top_states)]


In [8]:
hacked_rows = per_capita_state_subset[per_capita_state_subset.hacked]
assert max(hacked_rows.date) <= pd.Timestamp('2020-03-15')

In [9]:

positive = alt.Chart().mark_line(strokeDash=[10,10]).encode(
    alt.X('date'),
    alt.Y('positive', scale=alt.Scale(type='log'), title="per capita"),
)
total = alt.Chart().mark_line().encode(
    alt.X('date'),
    alt.Y('total', scale=alt.Scale(type='log')),
)

title = ["new tests (solid line)",
         "new positive results (dashed line)",
        "(source code: https://github.com/dchudz/something)",
        f"showing the {n_states} states with most total tests"]

alt.layer(positive, total
         ).facet(
    data=per_capita_state_subset,
    facet=alt.Facet('state', title=title),
                columns=3).configure_legend(symbolSize=120,
                                   labelLimit= 0,
                                   symbolType='circle',
                                   labelFontSize=14
                )

alt.FacetChart(...)

In [10]:
us_totals = states_new_df.set_index(['state', 'date'])[['positive', 'total']].groupby('date').sum().reset_index()
us_totals['rate'] = us_totals.positive/us_totals.total

In [11]:
us_totals = us_totals[1:] # remove day with zero

In [12]:
positive = alt.Chart().mark_line(strokeDash=[10,10]).encode(
    alt.X('date'),
    alt.Y('positive', scale=alt.Scale(type='log')),
)

total = alt.Chart().mark_line().encode(
    alt.X('date'),
    alt.Y('total', scale=alt.Scale(type='log')),
)

rate = alt.Chart(us_totals, title="US: (positive tests)/(total tests)",).mark_line().encode(
    alt.X('date'),
    alt.Y('rate', scale=alt.Scale(type='log')),
)

alt.vconcat(alt.layer(positive, total, data=us_totals, title="US: total tests (solid), positive tests (dotted)"), 
            rate)


alt.VConcatChart(...)